# MA-ES Template

## Imports

In [1]:
from pckgConOpt.maes import MAES, epsMAgES, keep_range, repair, eps_rnk, lex_rnk
import numpy as np
import seaborn as sea
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets, interact_manual
from pckgConOpt import tuttestfunctions as ttf
from pckgConOpt import o1,o2,o3,o4

## Helper methods

In [2]:
# no bound constraint handling
def noboundhand(y,lb, ub):
    return y

def norepair(y,max_reps,fevals,gv,hv,convio,objFun,lower_bounds,upper_bounds,delt,dim,g):
    return y

def fit_rnk(newpop_f,newpop_cv,eps):
    return np.argsort(newpop_f)

In [3]:
def print_results(name, gb_fv, dyn_cv, dyn_fev):
    print('objective function: {0}'.format(name))
    print('best fitness: {0:.4g}'.format(gb_fv))
    print('best constraint violation: {0:.4g}'.format(dyn_cv[-1]))
    print('number of function evaluations: {0:d}'.format(dyn_fev[-1]))

In [4]:
def visualize_single_run_statistics(dyn_gen, dyn_fit, dyn_cv, dyn_sig, name):
    # visualize single run statistics
    fig, axs = plt.subplots(1,3,figsize=(15,4))
    
    cax = axs[0]
    sea.scatterplot(x=dyn_gen, y = dyn_fit, legend='full', ax=cax)
    cax.set_xscale('log')
    #cax.set_yscale('log')
    cax.set_title('Fitness')
    
    cax = axs[1]
    sea.scatterplot(x=dyn_gen, y = dyn_cv, legend='full', ax=cax)
    cax.set_xscale('log')
    cax.set_yscale('log')
    cax.set_title('Constraint Violation')
    
    cax = axs[2]
    sea.scatterplot(x=dyn_gen, y = dyn_sig, legend='full', ax=cax)
    cax.set_xscale('log')
    cax.set_yscale('log')
    cax.set_title('Mutation Strength')

    fig.suptitle(name)
    plt.show()

In [5]:
def set_bounds(objFun, d):
    if objFun == ttf.cop3:
        lower_bounds = -20*np.ones(shape=(d,))
        upper_bounds = 20*np.ones(shape=(d,))
    elif objFun == ttf.cop5:
        lower_bounds = np.array([2.6, 0.7, 17, 7.3, 7.3, 2.9, 5])
        upper_bounds = np.array([3.6, 0.8, 28, 8.3, 8.3, 3.9, 5.5])
    else:
        lower_bounds = -100*np.ones(shape=(d,))
        upper_bounds = 100*np.ones(shape=(d,))
    
    return lower_bounds, upper_bounds

## Parameters

There are different **Constrained Optimization Problems (COP)** available (1-5)
- COP 1: ... 
- COP 2: ...
- COP 3: ...
- COP 4: ...
- COP 5: ...

#### Problem Dimension (d)
... 

#### Population Parameters 
- mu: ..
- lam: ..
- sigma: ..

#### Bound Constraint Handling (bch)
- No Bound Constraint Handling: ...
- Keep Range: ...

#### Gradient based Repair (rep)
- No Repair: ...
- Gradient based Repair: ...

In [11]:
def run(d, mu, lam, sigma, problem, bch,rep,rnk):   
    # seed random number generator
    default_rng = np.random.default_rng(123345)

    # run parameters
    maxIter = 10**3
    budget = maxIter*lam

    if problem == 'All':
        functions = [ttf.cop1, ttf.cop2, ttf.cop3, ttf.cop4, ttf.cop5]
    elif problem == 'Cop1': 
        functions = [ttf.cop1]
    elif problem == 'Cop2':
        functions = [ttf.cop2]
    elif problem == 'Cop3':
        functions = [ttf.cop3]
    elif problem == 'Cop4':
        functions = [ttf.cop4]
    elif problem == 'Cop5':
        d = 7
        functions = [ttf.cop5]

    if bch == "Keep Range":
        bch= keep_range
    else: 
        bch = noboundhand

    if rep == "Repair":
        rep = repair
    else: 
        rep = norepair

    if rnk == "Eps":
        rnk = eps_rnk
    elif rnk == "Lex":
        rnk = lex_rnk
    else: 
        rnk = fit_rnk
        
    # problems
    for objFun in functions:
        lower_bounds, upper_bounds = set_bounds(objFun, d)

        # random initial point
        x_init = lower_bounds + (upper_bounds - lower_bounds)*default_rng.uniform(size=(d,))

        # run MA-ES
        out = MAES(x_init,mu,lam,sigma,maxIter,budget, objFun, lower_bounds, upper_bounds, bch, rep, rnk)
        gb_y, gb_fv, dyn_gen, dyn_fev, dyn_fit, dyn_cv,  dyn_sig, dyn_ynorm = out

        print_results(objFun.__name__, gb_fv, dyn_cv, dyn_fev)
        visualize_single_run_statistics(dyn_gen, dyn_fit, dyn_cv, dyn_sig, objFun.__name__)

In [12]:
align_kw = {
    #"style": {"description_width": "400px"},  
    #"layout": widgets.Layout(width="500px")
}

im = interact_manual.options(manual_name = "Calculate")

im(run, 
         d = widgets.IntSlider(min = 0, max = 30, value = 10, **align_kw),
         mu = widgets.IntSlider(min = 0, max = 10, value = 3, description = "μ", **align_kw),
         lam = widgets.IntSlider(min = 0, max = 30, value = 10, description = "λ", **align_kw),
         sigma = widgets.FloatSlider(min = 0., max = 2., value = 1., description = "σ", **align_kw),
         problem = widgets.Dropdown(options = ['Cop1', 'Cop2', 'Cop3', 'Cop4', 'Cop5', 'All'], description = "Problem", **align_kw),
         rnk = widgets.Dropdown(options = ['Fitness','Lex','Eps'], description = "RNK", **align_kw),
         bch = widgets.Dropdown(options = ['None','Keep Range'], description = "BCH", **align_kw),
         rep = widgets.Dropdown(options = ['None','Repair'], description = "REP", **align_kw)
         )

interactive(children=(IntSlider(value=10, description='d', max=30), IntSlider(value=3, description='μ', max=10…

<function __main__.run(d, mu, lam, sigma, problem, bch, rep, rnk)>